# Part 2: Convolutional Neural Network





## A) All layers in the VGG-19 

<i style="color:green"> Required libraries are imported </i>

In [1]:
from google.colab import drive
import os
from torch.utils.data import Dataset, DataLoader
import glob
import torch
import cv2
import torchvision
import torch.optim as optim

drive.mount('/content/drive/')

Mounted at /content/drive/


<i style="color:green"> Getting foldernames from dataset </i>

In [2]:
class CustomDataset(Dataset):
  def __init__(self):
    self.base_path = "/content/drive/MyDrive/raw-img/"
    file_list = glob.glob(self.base_path + "*")
    self.data = []
    for class_path in file_list:
      class_name = class_path.split("/")[-1]
      for img_path in glob.glob(class_path + "/*.jpeg"):
        self.data.append([img_path, class_name])
        self.class_map = {"cane": 0, "cavallo": 1, "elefante": 2, "farfalla": 3, "gallina": 4, "gatto": 5, "mucca": 6,
                    "pecora": 7, "ragno": 8, "scoiattolo": 9}
        self.img_dim = (224, 224)
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    img_path, class_name = self.data[idx]
    img = cv2.imread(img_path)
    img = cv2.resize(img, self.img_dim)
    label = self.class_map[class_name]
    img_tensor = torch.from_numpy(img)
    img_tensor = img_tensor.float()
    img_tensor = img_tensor.permute(2, 0, 1)
    label = torch.tensor(label)
    return img_tensor, label

In [3]:
dataset = CustomDataset()

In [4]:
print(len(dataset))

24209


In [5]:
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*8/10), len(dataset)-int(len(dataset)*8/10)])

In [6]:
train_data_loader = torch.utils.data.DataLoader(
       train_set, batch_size=32,
       shuffle=True
   )

test_data_loader = torch.utils.data.DataLoader(
       test_set, batch_size=32,
       shuffle=True
   )

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Implementing VGG19 Algorithm

In [8]:
vgg_based = torchvision.models.vgg19(pretrained=True)

for param in vgg_based.parameters():
  param.requires_grad = False

# Modify the last layer
number_features = vgg_based.classifier[6].in_features
features = list(vgg_based.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(number_features, 10)])
vgg_based.classifier = torch.nn.Sequential(*features)

vgg_based = vgg_based.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_based.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

### Definition of Train function

In [9]:
def train_model(model, criterion, optimizer, num_epochs=10):
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    train_loss = 0
    for i, data in enumerate(train_data_loader):
      inputs , labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        outputs  = model(inputs)
        loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * inputs.size(0)
      print('{} Ara Loss: {:.4f}'.format('train', train_loss / len(train_set)))
    print('{} Loss: {:.4f}'.format(
               'train', train_loss / len(train_set)))
  return model

### Training Model (VGG19)

In [10]:
vgg_based = train_model(vgg_based, criterion, optimizer_ft, num_epochs=10)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
train Ara Loss: 54.1495
train Ara Loss: 54.2564
train Ara Loss: 54.3873
train Ara Loss: 54.5025
train Ara Loss: 54.5689
train Ara Loss: 54.7063
train Ara Loss: 54.7687
train Ara Loss: 54.8713
train Ara Loss: 54.9964
train Ara Loss: 55.0701
train Ara Loss: 55.1627
train Ara Loss: 55.2795
train Ara Loss: 55.3758
train Ara Loss: 55.4763
train Ara Loss: 55.5505
train Ara Loss: 55.6466
train Ara Loss: 55.7237
train Ara Loss: 55.8425
train Ara Loss: 55.9634
train Ara Loss: 56.0886
train Ara Loss: 56.2187
train Ara Loss: 56.3606
train Ara Loss: 56.5192
train Ara Loss: 56.6545
train Ara Loss: 56.7865
train Ara Loss: 56.9305
train Ara Loss: 57.0267
train Ara Loss: 57.1548
train Ara Loss: 57.2410
train Ara Loss: 57.3326
train Ara Loss: 57.4615
train Ara Loss: 57.5732
train Ara Loss: 57.6349
train Ara Loss: 57.7847
train Ara Loss: 57.8998
train Ara Loss: 58.0316
train Ara Loss: 58.1287
train Ara Loss: 58.2249
train Ara Loss: 58.3166
train Ara Loss: 5

In [11]:
def test_model(model):
  print("this is test")

In [12]:
vgg_based

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

### Implementation of the optimization algorithm

In [13]:
import torch.optim as optim
from torch import nn 
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.SGD(vgg_based.parameters(), lr=0.001)

In [14]:
train_on_gpu = torch.cuda.is_available()
import numpy as np

### Implemention of Test function

In [15]:
def seq (model, df, name ): 
    train_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for batch_i, (data, target) in enumerate(df):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            model.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        if name == 'train': 
            loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        _, pred = torch.max(output, 1) 
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        
    return class_correct, class_total, train_loss

### Definition of Labels

In [20]:
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly",
             "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", 
             "ragno": "spider", "scoiattolo": "squirrel" }
classes = ["cane", "cavallo", "elefante", "farfalla", "gallina", "gatto", "mucca", "pecora","ragno", "scoiattolo" ]


### Definition of the function that prints the results

In [21]:
def printdata(class_correct, class_total, train_loss, epoch, name, df ): 
    print(f'Epoch %d, loss: %.8f \t{name} Accuracy (Overall): %2d%% (%2d/%2d)' %(epoch,
        train_loss / len(df), 100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    if ((epoch+1) % 5 == 0 or epoch == 1):
        for i in range(10):
            if class_total[i] > 0:
                print(f'{name} Accuracy of %5s: %2d%% (%2d/%2d)' % (
                translate[classes[i]], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))

In [22]:
 # track test loss 
# over 10 animals classes
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
vgg_based.eval()
class_correct, class_total, train_loss= seq(vgg_based, test_data_loader, 'test')
printdata(class_correct, class_total, train_loss, 1, 'test', test_data_loader)

Epoch 1, loss: 32.72482463 	test Accuracy (Overall): 54% (2624/4842)
test Accuracy of   dog: 68% (646/944)
test Accuracy of horse: 39% (214/541)
test Accuracy of elephant: 39% (89/223)
test Accuracy of butterfly: 67% (225/332)
test Accuracy of chicken: 67% (441/649)
test Accuracy of   cat: 22% (55/241)
test Accuracy of   cow: 60% (227/376)
test Accuracy of sheep: 35% (106/295)
test Accuracy of spider: 67% (593/882)
test Accuracy of squirrel:  7% (28/359)


We used the VGG19 artificial neural network in part A of the 2nd part of the project. Our dataset contains data from 10 different animals. We used the mini-batch technique for training, thus overcoming the disadvantages of the size of our dataset. Unlike part B, we trained all layers in the training and obtained the following accuracy values ​​for different classes in the dataset.

Accuracy of dog: 68% (646/944) <br>
Accuracy of horse: 39% (214/541) <br>
Accuracy of elephant: 39% (89/223) <br>
Accuracy of butterfly: 67% (225/332) <br>
Accuracy of chicken: 67% (441/649) <br>
Accuracy of cat: 22% (55/241) <br>
Accuracy of cows: 60% (227/376) <br>
Accuracy of sheep: 35% (106/295) <br>
Accuracy of spider: 67% (593/882) <br>
Accuracy of squirrel: 7% (28/359) <br>

We achieved higher accuracy values ​​in dog, butterfly, chicken classes compared to others, but especially in squirrel, the accuracy value was noticeably low. This is because squirrels are often treated as cat or bird. Trained Neural Network failed to understand this difference. One reason for this is the indiscriminate samples in the data set.